# Part III
Using the previous two tutorials, please answer the following using an encorder-decoder approach and an LSTM compared approach.

Please create a transformer-based classifier for English name classification into male or female.

There are several datasets for name for male or female classification. In subseuqent iterations, this could be expanded to included more classifications.

Below is the source from NLTK, which only has male and female available but could be used for the purposes of this assignment.

```
names = nltk.corpus.names
names.fileids()
['female.txt', 'male.txt']
male_names = names.words('male.txt')
female_names = names.words('female.txt')
[w for w in male_names if w in female_names]
['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis',
'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel',
'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', ...]
```

In [1]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
import nltk
from nltk.corpus import names
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Download NLTK resources
nltk.download('names')

# Step 1: Prepare the dataset
male_names = [name.lower() for name in names.words('male.txt')]
female_names = [name.lower() for name in names.words('female.txt')]

# Step 2: Preprocess the data
all_names = male_names + female_names
max_length = max(len(name) for name in all_names)

# Tokenize names
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(all_names)
vocab_size = len(tokenizer.word_index) + 1

# Step 3: Prepare inputs and targets
# Create sequences of tokens
sequences = tokenizer.texts_to_sequences(all_names)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Create labels (0 for male, 1 for female)
labels = np.array([0] * len(male_names) + [1] * len(female_names))

# Step 4: Build and train the transformer-based classifier model
# Define model architecture (Transformer)
class TransformerClassifier(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(TransformerClassifier, self).__init__()
        self.encoder = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.num_layers = num_layers
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.flatten = tf.keras.layers.Flatten()
        self.dropout = tf.keras.layers.Dropout(rate)
        self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x, training, mask=None):
        seq_len = tf.shape(x)[1]
        x = self.encoder(x)
        x *= tf.math.sqrt(tf.cast(d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# Define positional encoding
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

# Define Transformer encoder layer
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        attn_output = self.mha(x, x, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

# Initialize model parameters
num_layers = 4
d_model = 128
num_heads = 8
dff = 512
input_vocab_size = vocab_size
maximum_position_encoding = max_length

# Build and compile the model
transformer_classifier = TransformerClassifier(num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding)
transformer_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Train the model
transformer_classifier.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Step 5: Evaluate the model on test data
loss, accuracy = transformer_classifier.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

### END CODE HERE ###

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


Epoch 1/10
100/100 [==============================] - 69s 609ms/step - loss: 0.7548 - accuracy: 0.5976 - val_loss: 0.6956 - val_accuracy: 0.6092
Epoch 2/10
100/100 [==============================] - 58s 582ms/step - loss: 0.6747 - accuracy: 0.6156 - val_loss: 0.6699 - val_accuracy: 0.6092
Epoch 3/10
100/100 [==============================] - 57s 573ms/step - loss: 0.6723 - accuracy: 0.6159 - val_loss: 0.6748 - val_accuracy: 0.6092
Epoch 4/10
100/100 [==============================] - 58s 582ms/step - loss: 0.6848 - accuracy: 0.5976 - val_loss: 0.6766 - val_accuracy: 0.6092
Epoch 5/10
100/100 [==============================] - 58s 577ms/step - loss: 0.6722 - accuracy: 0.6200 - val_loss: 0.6785 - val_accuracy: 0.6092
Epoch 6/10
100/100 [==============================] - 57s 575ms/step - loss: 0.6706 - accuracy: 0.6197 - val_loss: 0.6851 - val_accuracy: 0.6092
Epoch 7/10
100/100 [==============================] - 57s 567ms/step - loss: 0.6755 - accuracy: 0.6217 - val_loss: 0.6713 - val_ac

# References
1. https://arxiv.org/pdf/2102.03692.pdf
2. https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/exercise/13-attention.html
3. https://towardsdatascience.com/deep-learning-gender-from-name-lstm-recurrent-neural-networks-448d64553044
4. https://www.nltk.org/book/ch02.html#sec-lexical-resources